Import Libraries

In [4]:
import numpy as np
import pandas as pd
import spacy
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.utils import np_utils

In [5]:
print(tf.__version__)

2.2.0


Loading Features

In [6]:
from numpy import genfromtxt
features = genfromtxt('features.csv', delimiter=',')
encoded_labels = genfromtxt('classes.csv', delimiter=',')

In [7]:
encoded_labels = encoded_labels.astype('int64')

Splitting Our DATA

In [8]:
split_frac = 0.8

## split data into training, validation, and test data (features and labels, x and y)

split_idx = int(len(features)*split_frac)
train_x, remaining_x = features[:split_idx], features[split_idx:]
train_y, remaining_y = encoded_labels[:split_idx], encoded_labels[split_idx:]

test_idx = int(len(remaining_x)*0.5)
val_x, test_x = remaining_x[:test_idx], remaining_x[test_idx:]
val_y, test_y = remaining_y[:test_idx], remaining_y[test_idx:]

## print out the shapes of your resultant feature data
print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), "Train y: \t\t{}".format(train_y.shape),
      "\nValidation set: \t{}".format(val_x.shape),"valid y: \t\t{}".format(val_y.shape),
      "\nTest set: \t\t{}".format(test_x.shape),"Test y: \t\t{}".format(test_y.shape)) 

			Feature Shapes:
Train set: 		(11656, 15) Train y: 		(11656, 3) 
Validation set: 	(1457, 15) valid y: 		(1457, 3) 
Test set: 		(1457, 15) Test y: 		(1457, 3)


Importing Layers from Keras

In [9]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import SpatialDropout1D
from keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

In [10]:
no_of_words = 9481

**MODEL 1:**


In [11]:
model1 = Sequential()
model1.add(Embedding(no_of_words, 100,input_length=15))
#model1.add(SpatialDropout1D(0.2))
model1.add(LSTM(128))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(256, activation='relu'))
model1.add(Dropout(0.5))
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(3, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
filepath = "model.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True)
callbacks_list = [checkpoint]

model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 100)           948100    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)              

**MODEL 1: TRAIN**

In [12]:
batch_size=20
print('Train...')
model1.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=10,
          validation_data=(val_x, val_y),verbose=1)


Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11656 samples, validate on 1457 samples
Epoch 1/10
11656/11656 [==============================] - 34s 3ms/step - loss: 1.1181 - accuracy: 0.5728 - val_loss: 0.6637 - val_accuracy: 0.7097
Epoch 2/10
11656/11656 [==============================] - 31s 3ms/step - loss: 0.6435 - accuracy: 0.7332 - val_loss: 0.6016 - val_accuracy: 0.7536
Epoch 3/10
11656/11656 [==============================] - 31s 3ms/step - loss: 0.5049 - accuracy: 0.7994 - val_loss: 0.6060 - val_accuracy: 0.7639
Epoch 4/10
11656/11656 [==============================] - 32s 3ms/step - loss: 0.4304 - accuracy: 0.8345 - val_loss: 0.6534 - val_accuracy: 0.7502
Epoch 5/10
11656/11656 [==============================] - 32s 3ms/step - loss: 0.3883 - accuracy: 0.8527 - val_loss: 0.6262 - val_accuracy: 0.7591
Epoch 6/10
11656/11656 [==============================] - 31s 3ms/step - loss: 0.3549 - accuracy: 0.8675 - val_loss: 0.6391 - val_accuracy: 0.7502
Epoch 7/10
11656/11656 [==============================] - 32s 3ms/ste

**MODEL 1 : EVALUATE**

In [13]:

score, acc = model1.evaluate(test_x, test_y,
                            batch_size=batch_size)
print(score,acc)

1457/1457 [==============================] - 1s 347us/step
0.8551346178568628 0.7268359661102295
